<a href="https://colab.research.google.com/github/Rashiduzzaman2655/Numerical-Method-Fall2020/blob/main/Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Newton divided difference interpolation method:**

In [14]:
def proterm(i, value, x): 
	pro = 1; 
	for j in range(i): 
		pro = pro * (value - x[j]); 
	return pro; 

def dividedDiffTable(x, y, n): 

	for i in range(1, n): 
		for j in range(n - i): 
			y[j][i] = ((y[j][i - 1] - y[j + 1][i - 1]) /
									(x[j] - x[i + j])); 
	return y; 

def applyFormula(value, x, y, n): 

	sum = y[0][0]; 

	for i in range(1, n): 
		sum = sum + (proterm(i, value, x) * y[0][i]); 
	
	return sum; 

def printDiffTable(y, n): 

	for i in range(n): 
		for j in range(n - i): 
			print(round(y[i][j], 4), "\t", 
							end = " "); 

		print(""); 

n = 4; 
y = [[0 for i in range(10)] 
		for j in range(10)]; 
x = [ 5, 6, 9, 11 ]; 


y[0][0] = 12; 
y[1][0] = 13; 
y[2][0] = 14; 
y[3][0] = 16; 


y=dividedDiffTable(x, y, n); 
 
printDiffTable(y, n); 

value = 7; 

print("\nValue at", value, "is", 
		round(applyFormula(value, x, y, n), 2)) 





12 	 1.0 	 -0.1667 	 0.05 	 
13 	 0.3333 	 0.1333 	 
14 	 1.0 	 
16 	 

Value at 7 is 13.47


# "Maximum and Minimum interpolation formoula"

In [22]:
import numpy as np
L = np.random.random(100)
sum(L)
np.sum(L)
big_array = np.random.rand(1000000)
%timeit sum(big_array)
%timeit np.sum(big_array)
np.min(big_array), max(big_array)
np.min(big_array), np.max(big_array)
%timeit min(big_array)
%timeit np.min(big_array)
print(big_array.min(), big_array.max(), big_array.sum())
M = np.random.random((3, 4))
print(M)
M.sum()
M.min(axis=0)
M.max(axis=1)
data=[189, 170, 189, 163, 183]
heights = np.array(data['height(cm)'])
print(heights)
heights = np.array(data['height(cm)'])
print(heights)
print("Minimum height:    ", heights.min())
print("Maximum height:    ", heights.max())

10 loops, best of 3: 181 ms per loop
1000 loops, best of 3: 370 µs per loop
10 loops, best of 3: 107 ms per loop
1000 loops, best of 3: 425 µs per loop
1.5246792927081643e-06 0.9999994945851726 500291.2681677153
[[0.95884823 0.14226662 0.91687131 0.4585458 ]
 [0.26191873 0.52637772 0.07573245 0.25379603]
 [0.77557934 0.16976231 0.62046144 0.41245066]]


### **"Implement Factorization Method"**

In [11]:
import numpy as np


class MF():
    
    def __init__(self, R, K, alpha, beta, iterations):
        
        
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))
        
        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])
        
        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]
        
        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 10 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))
        
        return training_process

    def mse(self):
        
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)
            
            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])
            
            # Create copy of row of P since we need to update it but use older values for update on Q
            P_i = self.P[i, :][:]
            
            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * P_i - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction
    
    def full_matrix(self):
       
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)